In [1]:
    # Celda 1: Importaciones y carga de variables de entorno desde .env
    import os
    import sys
    import pandas as pd
    import pyodbc
    from dotenv import load_dotenv 

    # -------- CÓDIGO PARA CARGAR .env --------
    # Imprime el directorio de trabajo actual para depuración
    current_working_directory = os.getcwd()
    print(f"Directorio de trabajo actual del kernel: {current_working_directory}")

    # Asumimos que el archivo .env está en la RAÍZ del proyecto VOD-MARKETING-LAB
    # Si el kernel se inicia en la carpeta 'notebooks', subimos un nivel.
    # Si el kernel se inicia en la raíz del proyecto, no subimos.

    # Intenta determinar la raíz del proyecto.
    # Si el nombre de la carpeta actual es 'notebooks', asumimos que estamos dentro de ella.
    if os.path.basename(current_working_directory) == 'notebooks':
        project_root = os.path.abspath(os.path.join(current_working_directory, '..'))
    else:
        # Si no, asumimos que el CWD ya es la raíz del proyecto (o necesitamos ajustarlo)
        project_root = current_working_directory 
        # Si sospechas que CWD no es la raíz, podrías hardcodear la ruta a la raíz del proyecto aquí
        # como fallback, pero es mejor que el CWD sea predecible.
        # Ejemplo de fallback (ajusta a tu estructura):
        # project_root = "/home/diegokernel/vod-marketing-lab" 

    print(f"Raíz del proyecto determinada como: {project_root}")
    
    dotenv_path = os.path.join(project_root, '.env')

    if os.path.exists(dotenv_path):
        load_dotenv(dotenv_path=dotenv_path)
        print(f"Variables de entorno cargadas desde: {dotenv_path}")
    else:
        print(f"Archivo .env NO encontrado en: {dotenv_path}.")
        print("Asegúrate de que el archivo '.env' exista en la raíz de tu proyecto (ej. /home/diegokernel/vod-marketing-lab/.env)")
        print("Y que la 'Raíz del proyecto determinada como:' sea correcta.")
    # -------- FIN CÓDIGO PARA CARGAR .env --------

    # Añadir la carpeta raíz del proyecto al sys.path para encontrar el módulo utils
    # (Esta parte es para importar tu módulo utils.db)
    if project_root not in sys.path:
        sys.path.insert(0, project_root)

    try:
        from utils.db import get_sqlserver_conn
        print("Módulo utils.db importado correctamente.")
    except ImportError as e:
        print(f"Error importando utils.db: {e}")
        print(f"Buscando utils en sys.path que incluye: {project_root}")
    
    # Obtener credenciales (ya deberían estar cargadas si .env funcionó)
    azure_server = os.getenv("AZURE_SQL_SERVER")
    azure_database = os.getenv("AZURE_SQL_DATABASE")
    azure_user = os.getenv("AZURE_SQL_USER")
    azure_password = os.getenv("AZURE_SQL_PASSWORD")

    print(f"Servidor Azure: {azure_server}")
    print(f"Base de Datos Azure: {azure_database}")
    print(f"Usuario Azure: {azure_user}")
    print(f"Contraseña Azure LEÍDA POR OS.GETENV: '{azure_password}'") # <--- AÑADE ESTE PRINT

    

Directorio de trabajo actual del kernel: /home/diegokernel/vod-marketing-lab
Raíz del proyecto determinada como: /home/diegokernel/vod-marketing-lab
Variables de entorno cargadas desde: /home/diegokernel/vod-marketing-lab/.env
Módulo utils.db importado correctamente.
Servidor Azure: vod-marketing-server.database.windows.net
Base de Datos Azure: vod_marketing_db
Usuario Azure: dbadmin
Contraseña Azure LEÍDA POR OS.GETENV: '@Infernity1'


In [2]:
# Celda 2 (o siguiente celda): Conectar y ejecutar consulta
conn = None
if all([azure_server, azure_database, azure_user, azure_password]):
    try:
        print(f"Intentando conectar a Azure SQL: {azure_server}...")
        conn = get_sqlserver_conn(
            server=azure_server,
            database=azure_database,
            user=azure_user,
            pwd=azure_password
        )
        print("¡Conexión a Azure SQL exitosa!")

        # IMPORTANTE: Reemplaza esta consulta con una que funcione para una tabla real
        # que exista en tu base de datos Azure SQL (vod_marketing_db).
        # Por ejemplo, si tienes una tabla llamada 'Clientes':
        # query = "SELECT TOP 5 * FROM dbo.Clientes;"
        # O para ver algunas tablas del sistema si no tienes tablas de usuario aún:
        query = "SELECT TOP 5 TABLE_NAME, TABLE_SCHEMA, TABLE_TYPE FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        
        print(f"Ejecutando consulta: {query}")
        df = pd.read_sql(query, conn)
        
        print("Resultados de la consulta:")
        if not df.empty:
            display(df) # 'display()' funciona bien en Jupyter para DataFrames
        else:
            print("La consulta no devolvió resultados.")

    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error de pyodbc conectando o consultando Azure SQL: {sqlstate}")
        print(f"Detalles del error: {ex}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if conn:
            conn.close()
            print("Conexión a Azure SQL cerrada.")
else:
    print("Una o más variables de entorno para Azure SQL no están definidas. Verifica tu archivo .env o la configuración del entorno.")
    print("Asegúrate de haber ejecutado la celda anterior que carga las variables desde .env (si usas ese método).")



Intentando conectar a Azure SQL: vod-marketing-server.database.windows.net...
DEBUG utils.db: Cadena de conexión construida: DRIVER={ODBC Driver 18 for SQL Server};SERVER=vod-marketing-server.database.windows.net;DATABASE=vod_marketing_db;UID=dbadmin;PWD=********;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;


¡Conexión a Azure SQL exitosa!
Ejecutando consulta: SELECT TOP 5 TABLE_NAME, TABLE_SCHEMA, TABLE_TYPE FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';
Resultados de la consulta:


/tmp/ipykernel_138385/1181772946.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TABLE_NAME,TABLE_SCHEMA,TABLE_TYPE
0,Orders,Sales,BASE TABLE
1,sales_orders,bronze,BASE TABLE
2,customers,bronze,BASE TABLE
3,marketing_campaigns,bronze,BASE TABLE


Conexión a Azure SQL cerrada.
